## Pulp Scheduling


In [102]:
from dataclasses import dataclass
from copy import deepcopy
import math
from typing import Tuple, List

import pulp

### Classes


In [103]:
@dataclass
class Topic:
    name: str
    hours: float
    
@dataclass
class Course:
    name: str
    topics: List[Topic]
    final_day: int
    relative_proportion: float

    def total_hours(self) -> float:
        """Sum of all the hours from each topic."""
        total = 0
        for topic in self.topics:
            total += topic.hours
        return total

### Functions


In [104]:
def round_to_nearest_half(number: float) -> float:
    return round(number * 2) / 2


def validate_courses(courses: Tuple[Course], hours_per_day: List[float]) -> None:
    total_proportion = 0
    n_days = len(hours_per_day)
    for course in courses:
        total_proportion += course.relative_proportion
        if course.final_day > n_days:
            raise ValueError(
                f"The final day of {course.name} ({course.final_day}) "
                f"is outside the number of days ({n_days})."
            )


    if total_proportion != 1:
        raise ValueError(f"Relative proportion did not sum to 1 ({total_proportion})")



def perform_course_scale(course: Course, factor: float) -> Course:
    topics = course.topics
    scaled_topics = []
    for topic in topics:
        scaled_hours = topic.hours * factor
        rounded_hours = round_to_nearest_half(scaled_hours)
        scaled_topic = Topic(topic.name, rounded_hours)
        scaled_topics.append(scaled_topic)

    scaled_course = Course(
        course.name, scaled_topics, course.final_day, course.relative_proportion
    )
    return scaled_course

def obtain_topic_hours(topic: Topic) -> float:
    return topic.hours

def enforce_hours(course: Course, required_hours: float) -> Course:
    enforced_course = deepcopy(course)
    topics = enforced_course.topics
    # Ordering topics from longest time to least
    ordered_topics = sorted(topics, key=obtain_topic_hours, reverse=True)
    current_hours = enforced_course.total_hours()

    i = 0
    while not (math.isclose(current_hours, required_hours)):
        index = i % len(topics)
        if current_hours < required_hours:
            ordered_topics[index].hours += 0.5  # Increase by 30 minutes if less than required
        else:
            ordered_topics[index].hours -= 0.5  # Decrease by 30 minutes if greater than required
        current_hours = enforced_course.total_hours()
        i += 1

    return enforced_course

def schedule_hour_split(courses: List[Course], total_study_hours: float) -> list[float]:
    hours_split = []
    for i in range(len(courses) - 1):
        course = courses[i]
        hours_to_study = round_to_nearest_half(course.relative_proportion * total_study_hours)
        hours_split.append(hours_to_study)

    # Adding the remaining time
    # Round to nearest 0.5
    hours_to_study = round_to_nearest_half(total_study_hours - sum(hours_split))
    hours_split.append(hours_to_study)

    return hours_split


def scale_courses(courses: List[Course], hours_per_day: List[int]) -> List[Course]:
    validate_courses(courses, hours_per_day)
    # Determining the number of hours required for each course for the given schedule
    total_study_hours = sum(hours_per_day)
    hours_split = schedule_hour_split(courses, total_study_hours)

    # Performing the rescale
    rescaled_courses = []
    for course, hours_to_study in zip(courses, hours_split):
        rescaling_factor = hours_to_study / course.total_hours()
        rescaled_course = perform_course_scale(course, rescaling_factor)
        rescaled_courses.append(rescaled_course)

    # Enforcing that the number of hours in each course matches the scaled hours
    enforced_courses = []
    for rescaled_course, hours_to_study in zip(rescaled_courses, hours_split):
        enforced_course = enforce_hours(rescaled_course, hours_to_study)
        enforced_courses.append(enforced_course)

    return enforced_courses

def obtain_final_day(course: Course) -> int:
    return course.final_day

def is_viable(courses: List[Course], hours_per_day: List[int]) -> bool:
    # Order courses by final day
    ordered_courses = sorted(courses, key=obtain_final_day)

    # Creating a list of the available number of hours for each course
    available_hours_list = []
    start_index = 0
    for course in ordered_courses:
        end_index = course.final_day + 1
        available_hours = sum(hours_per_day[start_index:end_index])
        available_hours_list.append(available_hours)
        start_index = end_index

    # Checking if the study schedule viable within the time constraints
    remainder = 0
    for i, course in enumerate(ordered_courses):
        available_hours_list[i] += remainder
        remainder = available_hours_list[i] - course.total_hours()
        if remainder < 0:
            # Study schedule is not viable
            return False
        
    # Study schedule is viable
    return True


def allocate_topics(
    courses: List[Course], hours_per_day: List[int]
) -> List[List[Topic]]:
    # Ensure immutability of original variables
    courses = deepcopy(courses)
    hours_per_day = hours_per_day.copy()

    # Order courses by their final day from closest to furthest away from start
    ordered_courses = sorted(courses, key=obtain_final_day)

    # Combine all topics into a single list
    topics = []
    for course in ordered_courses:
        for topic in course.topics:
            topics.append(deepcopy(topic))

    # Distribute topics to each day of the schedule
    schedule = [[] for _ in range(len(hours_per_day))]
    day = 0
    for topic in topics:
        while topic.hours > 0 and day < len(hours_per_day):
            if hours_per_day[day] > 0:
                hours_to_add = min(topic.hours, hours_per_day[day])
                schedule[day].append(Topic(topic.name, hours_to_add))
                topic.hours -= hours_to_add
                hours_per_day[day] -= hours_to_add
            if hours_per_day[day] == 0:
                day += 1

    return schedule

### Testing


In [105]:
# Inputs
# 1)

# Maths
linear_algebra = Topic("Linear Algebra", 1)
geometry = Topic("Geometry", 2)
differential_equations = Topic("Differential Equations", 4)
maths = Course("Maths", [linear_algebra, geometry, differential_equations], 1, 0.2)

# Physics
o_and_w = Topic("Oscillations and Waves", 3)
mechanics = Topic("Mechanics", 3)
quantum_physics = Topic("Quantum Physics", 4)
electromagnetism = Topic("Electromagnetism", 10)

physics = Course(
    "Physics", [o_and_w, mechanics, quantum_physics, electromagnetism], 6, 0.8
)

# Creating the schedule
courses = [maths, physics]
hours_per_day = [2, 4, 3, 1, 3, 0.5]  # Total = 13.5


scaled_courses = scale_courses(courses, hours_per_day)
print(is_viable(scaled_courses, hours_per_day))
daily_schedule = allocate_topics(scaled_courses, hours_per_day)

for day in daily_schedule:
    print(day)

True
[Topic(name='Linear Algebra', hours=0.5), Topic(name='Geometry', hours=0.5), Topic(name='Differential Equations', hours=1.0)]
[Topic(name='Differential Equations', hours=0.5), Topic(name='Oscillations and Waves', hours=1.5), Topic(name='Mechanics', hours=1.5), Topic(name='Quantum Physics', hours=0.5)]
[Topic(name='Quantum Physics', hours=1.5), Topic(name='Electromagnetism', hours=1.5)]
[Topic(name='Electromagnetism', hours=1)]
[Topic(name='Electromagnetism', hours=3)]
[Topic(name='Electromagnetism', hours=0.5)]
